# Zadatak

Na adresi https://www.foi.unizg.hr/hr/dokumenti nalazi se baza dokumenata Fakulteta organizacije i informatike. Većina dokumenata zadana je u PDF formatu, no neki od dokumenata su čitki (vektorski format) dok su drugi skenirani (rasterski format). Potrebno je:
1) Implementirati program koji će skinuti sve PDF dokumente na lokalno računalo (ili Google Colab / Drive direktorij)
2) Kreirati bazu podataka (SQLite) koja će sadržavati jednu tablicu "dokument". Tablica treba sadržavati šifru dokumenta (autonumber), naslov dokumenta, putanju datoteke (path i filename na lokalnom računalu), URL adresu s koje je dokument skinut, datum, te tekstualni sadržaj dokumenta.
3) Posebno za zadnje polje u tablici potrebno je svaki dokument učitati i ekstrahirati tekst (primjerice putem Python modula PDFMiner - https://github.com/pdfminer/pdfminer.six). Ukoliko je dokument skeniran potrebno je koristiti odgovarajući OCR modul za ekstrakciju teksta (npr. PyTesseract https://pypi.org/project/pytesseract/).
4) Omogućiti pristup skupu podataka putem REST API-ja koji omogućuje isključivo pretraživanje podataka putem:(a) pretraživanja ključnih riječi u tekstu (npr. boolean search), (b) pretraživanje ključnih riječi u naslovu (c) pregled prema datumu (npr. dokumenti od DATUM do DATUM), (d) izlistavanje svih dokumenata.

# Prikupljanje podataka

In [1]:
!mkdir -p data/pdfs data/dummy_data

In [128]:
import os
import sys
import csv
import datetime
import time

import requests
from bs4 import BeautifulSoup
from requests_html import HTMLSession

import numpy as np
import pandas as pd

from sqlalchemy import (create_engine, MetaData, Table, Column, Integer, Text,
                        String, Double, DateTime, insert, select, update, delete, text)

import fitz
import pytesseract
from PIL import Image
from io import BytesIO
from pdfminer.high_level import extract_text

In [47]:
s = HTMLSession()

def get_data(url, timeout=3, max_retries=3):
    retries = 0
    while retries < max_retries:
        try:
            response = s.get(url, timeout=timeout)
            soup = BeautifulSoup(response.text, 'html.parser')
            return soup
        except requests.exceptions.Timeout:
            print(f"Request timed out for URL: {url}. Retrying ({retries + 1}/{max_retries})...")
            retries += 1
        except requests.exceptions.RequestException as e:
            print(f"Error during request to {url}: {e}")
            break  # Break the loop on non-timeout errors

    print(f"Failed to fetch data from {url} after {max_retries} retries.")
    return None

## Downloading PDF

In [100]:
def download_pdf2(url, path, max_retries=3):
    '''
    Download a document from a given URL to a specified directory.
    URL format: /sites/default/files/[document_name].pdf
    '''
    file_path = None
    retries = 0
    while retries < max_retries:
        try:
            response = s.get(url, timeout = 3)
            response.raise_for_status()  # Checks for HTTP errors

            file_name = url.split('/')[-1]
            file_path = os.path.join(path, file_name)

            with open(file_path, 'wb') as f:
                f.write(response.content)

            print(f"Success: {file_path}")
            break  # Exit loop on successful download
        except requests.exceptions.RequestException as e:
            print(f"Attempt {retries + 1} failed: {e}")
            retries += 1

    if retries == max_retries:
        print(f"Failed to download file from {url} after {max_retries} retries.")

    return file_path

In [5]:
# fixed version
def download_pdf(url, path, max_retries=3, timeout=10):
    '''
    Download a document from a given URL to a specified directory.
    URL format: /sites/default/files/[document_name].pdf
    '''
    retries = 0
    backoff = 1  # Initial backoff duration in seconds

    while retries < max_retries:
        try:
            # Check response headers before downloading the full content
            with s.get(url, stream=True, timeout=timeout) as r:
                r.raise_for_status()  # Checks for HTTP errors

                file_name = url.split('/')[-1]
                file_path = os.path.join(path, file_name)

                with open(file_path, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        if chunk:  # Filter out keep-alive chunks
                            f.write(chunk)

            print(f"Success: {file_path}")
            break  # Exit loop on successful download
        except requests.exceptions.RequestException as e:
            print(f"Attempt {retries + 1} failed: {e}")
            sleep(backoff)  # Wait before retrying
            backoff *= 2  # Exponential backoff
            retries += 1

    if retries == max_retries:
        print(f"Failed to download file from {url} after {max_retries} retries.")

## Creating a CSV data entry/extracting useful fields

In [42]:
# fixed version
def get_metadata(url):
    '''
        Dohvaca meta podatke za zapis sa dane url adrese (/hr/dokument/[ime_dokumenta])
    '''
    soup = get_data(url)

    if not soup:
        return (None,) * 5

    ime_datoteke, link1, datum, vrsta_dokumenta, kategorija_dokumenta = (None,) * 5

    try:
        link_element = soup.find('a', href=True, type=lambda value: value and 'application/pdf' in value)
        if link_element:
            link1 = link_element['href']

        datum_element = soup.find('div', class_='datum')
        if datum_element:
            datum_text = datum_element.text.strip()
            datum = datum_text.split('Kreirano: ')[1] if 'Kreirano: ' in datum_text else None

        fields = soup.find_all('div', class_='field-item even')
        if fields:
            ime_datoteke = fields[0].text.strip() if len(fields) > 0 else None
            vrsta_dokumenta = fields[1].text.strip() if len(fields) > 1 else None
            kategorija_dokumenta = fields[2].text.strip() if len(fields) > 2 else None

    except Exception as e:
        print("Error while extracting data:", e)

    return ime_datoteke, link1, datum, vrsta_dokumenta, kategorija_dokumenta

In [43]:
metadata = get_metadata('https://www.foi.unizg.hr/hr/dokument/izmjenjena-odluka-pok-2023')
metadata

('odluka_o_izmjeni_odluke_pok_2023_10_12.pdf',
 'https://www.foi.unizg.hr/sites/default/files/odluka_o_izmjeni_odluke_pok_2023_10_12_0.pdf',
 '18.12.2023',
 'Odluka',
 'Kvaliteta')

## Handling pagination and going through document on the main pages

In [8]:
def extract_links(span_list):
    return [(span.find('a', href = True, target = False)['href'], span.text) for span in span_list]

In [9]:
soup = get_data('https://www.foi.unizg.hr/hr/dokumenti')
spans = soup.find_all('span', class_ = 'field-content') if soup else None

In [10]:
extract_links(spans)

[('/hr/dokument/izmjenjena-odluka-pok-2023', 'Izmjenjena odluka POK-a 2023.'),
 ('/hr/dokument/odluka-o-raspisivanju-izvanrednih-izbora-studentskog-zbora-suzgfoi',
  'Odluka o raspisivanju izvanrednih izbora Studentskog zbora SUZGFOI'),
 ('/hr/dokument/posebni-dio-financijskog-plana-2024-2026',
  'Posebni dio financijskog plana 2024-2026'),
 ('/hr/dokument/opci-dio-prijedlog-financijskog-plana-2024-2026',
  'Opći dio - prijedlog financijskog plana 2024-2026'),
 ('/hr/dokument/obrazlozenje-posebnog-dijela-financijskog-plana-2024-2026',
  'Obrazloženje posebnog dijela financijskog plana 2024-2026'),
 ('/hr/dokument/obrazlozenje-opceg-dijela-financijskog-plana-2024-2026',
  'Obrazloženje općeg dijela financijskog plana 2024-2026'),
 ('/hr/dokument/dnevni-red-fakultetskog-vijeca-07122023',
  'Dnevni red Fakultetskog vijeća 07.12.2023.'),
 ('/hr/dokument/politika-privatnosti-foi-ja', 'Politika privatnosti FOI-ja'),
 ('/hr/dokument/popis-gospodarskih-subjekata-s-kojima-se-ne-smije-sklapati-u

### Handling pagination

In [11]:
soup = get_data('https://www.foi.unizg.hr/hr/dokumenti')

In [12]:
def get_next_page(soup):
    root_url = 'https://www.foi.unizg.hr'
    page = soup.find('ul', class_ = 'pager')
    next_btn = page.find('li', class_ = 'pager-next')
    if next_btn:
        url = root_url + str(next_btn.find('a')['href'])
        return url
    else:
        return None

In [13]:
def get_urls(url):
    urls = []

    while True:

        soup = get_data(url)

        url = get_next_page(soup)
        if not url:
            break
    
        print(f'Appending {url}')
        urls.append(url)
        
    return urls

In [14]:
urls = get_urls('https://www.foi.unizg.hr/hr/dokumenti')

Appending https://www.foi.unizg.hr/hr/dokumenti?page=1
Appending https://www.foi.unizg.hr/hr/dokumenti?page=2
Appending https://www.foi.unizg.hr/hr/dokumenti?page=3
Appending https://www.foi.unizg.hr/hr/dokumenti?page=4
Appending https://www.foi.unizg.hr/hr/dokumenti?page=5
Appending https://www.foi.unizg.hr/hr/dokumenti?page=6
Appending https://www.foi.unizg.hr/hr/dokumenti?page=7
Appending https://www.foi.unizg.hr/hr/dokumenti?page=8
Appending https://www.foi.unizg.hr/hr/dokumenti?page=9
Appending https://www.foi.unizg.hr/hr/dokumenti?page=10
Appending https://www.foi.unizg.hr/hr/dokumenti?page=11
Appending https://www.foi.unizg.hr/hr/dokumenti?page=12
Appending https://www.foi.unizg.hr/hr/dokumenti?page=13
Appending https://www.foi.unizg.hr/hr/dokumenti?page=14
Appending https://www.foi.unizg.hr/hr/dokumenti?page=15
Appending https://www.foi.unizg.hr/hr/dokumenti?page=16
Appending https://www.foi.unizg.hr/hr/dokumenti?page=17
Appending https://www.foi.unizg.hr/hr/dokumenti?page=18
A

In [15]:
urls[:5]

['https://www.foi.unizg.hr/hr/dokumenti?page=1',
 'https://www.foi.unizg.hr/hr/dokumenti?page=2',
 'https://www.foi.unizg.hr/hr/dokumenti?page=3',
 'https://www.foi.unizg.hr/hr/dokumenti?page=4',
 'https://www.foi.unizg.hr/hr/dokumenti?page=5']

## Scraping main pages

In [203]:
def scrape_page(url):
    '''
        Prolazi kroz linkove glavne stranice, ekstrahira /hr/dokument/ linkove i
        dohvaca meta podatke za svaki dokument
    '''
    root_url = 'https://www.foi.unizg.hr'
    print(f'Scraping {url}')
    
    soup = get_data(url)
    spans = soup.find_all('span', class_ = 'field-content')

    links = extract_links(spans)

    metadata = [get_metadata(root_url + link[0]) for link in links]

    df1 = pd.DataFrame(metadata, columns = ['naziv_datoteke', 'url', 'datum', 'vrsta_dokumenta', 'kategorija_dokumenta'])
    df2 = pd.DataFrame(links, columns = ['metadata_link', 'naslov_dokumenta'])

    return pd.concat([df1, df2], axis = 1)

In [204]:
df = scrape_page('https://www.foi.unizg.hr/hr/dokumenti')

for url in urls:
    new_df = scrape_page(url)
    df = pd.concat([df, new_df], axis = 0, ignore_index = True)

Scraping https://www.foi.unizg.hr/hr/dokumenti
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=1
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=2
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=3
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=4
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=5
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=6
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=7
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=8
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=9
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=10
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=11
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=12
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=13
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=14
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=15
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=16
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=17
Scraping https://www.foi.un

In [205]:
# Sanity check
df.shape

(933, 7)

In [206]:
# Ciscenje dataframea
df.drop(['naziv_datoteke', 'vrsta_dokumenta', 'kategorija_dokumenta', 'metadata_link'], axis = 1, inplace = True)
df.dropna(subset = ['url'], inplace = True)
df.drop_duplicates(subset=['url'], inplace = True)
df['datum'] = pd.to_datetime(df['datum'], format = '%d.%m.%Y')
df.reset_index(drop = True, inplace = True)

In [207]:
df.shape

(849, 3)

In [208]:
df.head()

,url,datum,naslov_dokumenta
0,https://www.foi.unizg.hr/sites/default/files/o...,2023-12-18,Izmjenjena odluka POK-a 2023.
1,https://www.foi.unizg.hr/sites/default/files/o...,2023-12-12,Odluka o raspisivanju izvanrednih izbora Stude...
2,https://www.foi.unizg.hr/sites/default/files/f...,2023-12-07,Dnevni red Fakultetskog vijeća 07.12.2023.
3,https://www.foi.unizg.hr/sites/default/files/p...,2023-11-24,Politika privatnosti FOI-ja
4,https://www.foi.unizg.hr/sites/default/files/p...,2023-11-20,Popis gospodarskih subjekata s kojima se ne sm...


In [209]:
df.naslov_dokumenta.nunique()

831

Najbitniji su linkovi, datoteke kao dnevni red, odluke, modeli pracenja i sl. mogu imati iste nazive, ali pdf-ovi su naravno na drugim adresama

## Text & path pipeline

In [210]:
df_test = df.head().copy()
df_test

,url,datum,naslov_dokumenta
0,https://www.foi.unizg.hr/sites/default/files/o...,2023-12-18,Izmjenjena odluka POK-a 2023.
1,https://www.foi.unizg.hr/sites/default/files/o...,2023-12-12,Odluka o raspisivanju izvanrednih izbora Stude...
2,https://www.foi.unizg.hr/sites/default/files/f...,2023-12-07,Dnevni red Fakultetskog vijeća 07.12.2023.
3,https://www.foi.unizg.hr/sites/default/files/p...,2023-11-24,Politika privatnosti FOI-ja
4,https://www.foi.unizg.hr/sites/default/files/p...,2023-11-20,Popis gospodarskih subjekata s kojima se ne sm...


In [211]:
df_test['tekst'] = None
df_test['putanja'] = None

In [212]:
for i in range(len(df_test)):
    path = download_pdf2(df.iloc[i].url, './data/dummy_data')
    text = extract_text_from_pdf(path)
    df_test.at[i, 'tekst'] = text.strip()
    df_test.at[i, 'putanja'] = path

Success: ./data/dummy_data/odluka_o_izmjeni_odluke_pok_2023_10_12_0.pdf
Success: ./data/dummy_data/odluka-izvanredni_izbori_za_studentski_zbor-sijecanj_2024.pdf
Success: ./data/dummy_data/fv_dnevni_red_2023-12-07_3._sjednica.pdf
Success: ./data/dummy_data/politika-privatnosti-foi.pdf
Success: ./data/dummy_data/popis_gs_u_sukobu_interesa_2023.pdf


In [213]:
df_test

,url,datum,naslov_dokumenta,tekst,putanja
0,https://www.foi.unizg.hr/sites/default/files/o...,2023-12-18,Izmjenjena odluka POK-a 2023.,KLASA: 602-04/23-06/1 \nURBROJ: 2186-62-06-23-...,./data/dummy_data/odluka_o_izmjeni_odluke_pok_...
1,https://www.foi.unizg.hr/sites/default/files/o...,2023-12-12,Odluka o raspisivanju izvanrednih izbora Stude...,foj == =\n\nKLASA: 602-04/23-12/1\nURBROJ: 218...,./data/dummy_data/odluka-izvanredni_izbori_za_...
2,https://www.foi.unizg.hr/sites/default/files/f...,2023-12-07,Dnevni red Fakultetskog vijeća 07.12.2023.,SVEUČILIŠTE U ZAGREBU \nFAKULTET ORGANIZACIJE ...,./data/dummy_data/fv_dnevni_red_2023-12-07_3._...
3,https://www.foi.unizg.hr/sites/default/files/p...,2023-11-24,Politika privatnosti FOI-ja,VODITELJ OBRADE \n\nPOLITIKA PRIVATNOSTI \n\n...,./data/dummy_data/politika-privatnosti-foi.pdf
4,https://www.foi.unizg.hr/sites/default/files/p...,2023-11-20,Popis gospodarskih subjekata s kojima se ne sm...,foj s=> =\n\nKLASA: 303.02/23.02/2\n(URBROJ: 2...,./data/dummy_data/popis_gs_u_sukobu_interesa_2...


# SQLAlchemy

In [214]:
# Create an in-memory SQLite database
engine = create_engine('sqlite:///dokumenti.db')

metadata = MetaData()

# Define the "dokument" table with specified columns
dokument = Table('dokument', metadata,
                 Column('sifra_dokumenta', Integer, primary_key=True, autoincrement=True),
                 Column('url_adresa', String),
                 Column('datum', DateTime),
                 Column('naslov_dokumenta', String),
                 Column('tekstualni_sadrzaj', Text),
                 Column('putanja_datoteke', String))

# Create the table
metadata.create_all(engine)

In [215]:
# Checking the created table
dokument.columns.keys()

['sifra_dokumenta',
 'url_adresa',
 'datum',
 'naslov_dokumenta',
 'tekstualni_sadrzaj',
 'putanja_datoteke']

# PDF text extraction

In [145]:
def is_scanned_pdf(file_path):
    text = extract_text(file_path)
    return len(text.strip()) == 0

def extract_text_from_scanned_pdf(file_path):
    document = fitz.open(file_path)
    text = ""

    for page_num in range(len(document)):
        page = document.load_page(page_num)
        pix = page.get_pixmap()
        image = Image.open(BytesIO(pix.tobytes()))
        text += pytesseract.image_to_string(image, lang = 'hrv')

    document.close()
    return text

def extract_text_from_pdf(file_path):
    if is_scanned_pdf(file_path):
        return extract_text_from_scanned_pdf(file_path)
    else:
        return extract_text(file_path)